In [ ]:
!pip install transformers datasets

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving custom_data.txt to custom_data.txt


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Assign pad token
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [ ]:
with open("custom_data.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

print(lines[:5])  # preview the first 5 lines


['\n', 'Once upon a time in a land far away, there lived a curious little girl named Luna. She dreamed of flying among the stars.\n', '\n', 'The future of artificial intelligence lies in its ability to understand human emotions and make ethical decisions.\n', '\n']


In [ ]:
# Read the uploaded custom_data.txt
with open("custom_data.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Show the first 5 lines to verify the content
print("Sample lines from dataset:\n")
for i, line in enumerate(lines[:5], 1):
    print(f"{i}. {line.strip()}")


Sample lines from dataset:

1. 
2. Once upon a time in a land far away, there lived a curious little girl named Luna. She dreamed of flying among the stars.
3. 
4. The future of artificial intelligence lies in its ability to understand human emotions and make ethical decisions.
5. 


In [ ]:
from torch.utils.data import Dataset

class CustomTextDataset(Dataset):
    def __init__(self, lines, tokenizer, block_size=512):
        self.examples = []
        for line in lines:
            line = line.strip()
            if not line:
                continue
            tokens = tokenizer(
                line,
                truncation=True,
                padding="max_length",
                max_length=block_size,
                return_tensors="pt"
            )
            self.examples.append({
                "input_ids": tokens["input_ids"].squeeze(),
                "attention_mask": tokens["attention_mask"].squeeze(),
                "labels": tokens["input_ids"].squeeze(),
            })

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

# Create the dataset
train_dataset = CustomTextDataset(lines, tokenizer)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none",  # Disable wandb/logging for simplicity
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,6.392300


TrainOutput(global_step=15, training_loss=4.350615731875101, metrics={'train_runtime': 332.6096, 'train_samples_per_second': 0.09, 'train_steps_per_second': 0.045, 'total_flos': 7838760960000.0, 'train_loss': 4.350615731875101, 'epoch': 3.0})

In [ ]:

prompt = "The future of technology is"

inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    inputs["input_ids"],
    max_length=100,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,         # enables randomness
    top_k=50,                # limits to top k words
    top_p=0.95,              # nucleus sampling
    temperature=0.9          # creativity level
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("📝 Generated Text:\n")
print(generated_text)


📝 Generated Text:

The future of technology is becoming more like the sun of the future.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
save_path = "/content/drive/MyDrive/gpt2-finetuned-model"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/gpt2-finetuned-model/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2-finetuned-model/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2-finetuned-model/vocab.json',
 '/content/drive/MyDrive/gpt2-finetuned-model/merges.txt',
 '/content/drive/MyDrive/gpt2-finetuned-model/added_tokens.json')